In [1]:
import re
import os
import csv
import time
import urllib.request
from urllib.parse import quote
from user_agent import generate_user_agent

In [2]:
from win10toast import ToastNotifier

In [3]:
def html_from_keyword(keyword):
    url="https://www.google.com/search?q="+keyword+"&source=lnms&tbm=isch"
    request = urllib.request.Request(url,headers={"User-Agent": generate_user_agent()})
    page = urllib.request.urlopen(request)
    return str(page.read())

def get_image_link(keyword):
    page_content = html_from_keyword(keyword)
    link_list = re.findall('"ou":"(.*?)"', page_content)
    return set(link_list)

def download(keyword,prefix,suffix,img_dir,img_per_search=1,):
    name=keyword.replace(' ','_')
    if(prefix!=""):
        keyword=prefix+"+"+keyword
    if(suffix!=""):
        keyword=keyword+"+"+suffix
    keyword=keyword.replace(" ","+")
    image_links=set()
    attempt_count=0
    print("Entered download for",keyword)
    print("The link is","https://www.google.com/search?q="+keyword+"&source=lnms&tbm=isch")
    while(len(image_links)==0):
        image_links = image_links.union(get_image_link(keyword))
        attempt_count+=1
        if(attempt_count>200):
            print("Failed to find image links in 200 runs.")
            break
    print(keyword,"has",len(image_links),"image links")
    count=0
    for image_link in image_links:
        request = urllib.request.Request(image_link, headers = {"User-Agent": generate_user_agent()})
        count2=1
        moveOn=False
        while(True):
            try:
                response = urllib.request.urlopen(request)
                break
            except:
                if(count2==5):
                    #print("Request failed 5 times. Moving on to next link")
                    moveOn=True
                    break
                count2+=1
        if moveOn:
            continue
        data = response.read()
        #file_path = img_dir+keyword+str(count)+".png"
        file_path = img_dir+name+str(count)+".png"
        with open(file_path,'wb') as wf:
            wf.write(data)
            print("Image saved as",file_path)
        count+=1
        if(count==img_per_search):
            break

## Thousand images

In [4]:
crawlProgress=145 # Lang Rangler

In [5]:
toaster=ToastNotifier()

path = os.getcwd()+'\\thousand\\'
if not os.path.exists(path):
    os.mkdir(path)
    
names=[]
with open('jojo_list.txt') as file:
    data=file.read().splitlines()
    for name in data:
        append=True
        for enlisted in names:
            if name!=enlisted:
                short=name
                long=enlisted
                if len(short)>len(long):
                    temp=short
                    short=long
                    long=temp
#                 if short in long:
#                     print(short,"in",long)
#                     append=False
#                     break
            else:
                append=False
                break
        if(append):
            #print("appending",name)
            names.append(name)

print("There are",len(names),"distinct names")
print()

prefix="JoJo's+Bizarre+Adventure"
suffix="Anime+Face"
for i in range(crawlProgress,len(names)):
    if i%20==0 and i>0:
        print(str(i),"banes crawled.")
        #toaster.show_toast((str(i)+" names crawled."),"Yay!")
    name=names[i]
    while True:
        try:
            download(name,prefix,suffix,path,5)
            print()
            break
        except :
            crawlProgress=i
            print("Request Limit Reached Lol")
            print("We are at name",crawlProgress)
            # Wait for too many request thing to go away
            time.sleep(10*60)
toaster.show_toast("Crawling Done","Go check the images.")

There are 215 distinct names

Entered download for JoJo's+Bizarre+Adventure+Kenzou+Anime+Face
The link is https://www.google.com/search?q=JoJo's+Bizarre+Adventure+Kenzou+Anime+Face&source=lnms&tbm=isch
JoJo's+Bizarre+Adventure+Kenzou+Anime+Face has 100 image links
Image saved as C:\Users\andy5\Desktop\CS6103_Best_Project_2020\thousand\Kenzou0.png
Image saved as C:\Users\andy5\Desktop\CS6103_Best_Project_2020\thousand\Kenzou1.png
Image saved as C:\Users\andy5\Desktop\CS6103_Best_Project_2020\thousand\Kenzou2.png
Image saved as C:\Users\andy5\Desktop\CS6103_Best_Project_2020\thousand\Kenzou3.png
Image saved as C:\Users\andy5\Desktop\CS6103_Best_Project_2020\thousand\Kenzou4.png

Entered download for JoJo's+Bizarre+Adventure+D+an+G+Anime+Face
The link is https://www.google.com/search?q=JoJo's+Bizarre+Adventure+D+an+G+Anime+Face&source=lnms&tbm=isch
JoJo's+Bizarre+Adventure+D+an+G+Anime+Face has 100 image links
Image saved as C:\Users\andy5\Desktop\CS6103_Best_Project_2020\thousand\D_an_G0

True

In [6]:
print(crawlProgress)

202


In [12]:
import sys
import time
for i in range(10):
    sys.stdout.write("\r"+str(i))
    time.sleep(0.5)

9